In [1]:
from google import genai
from google.genai import types
client= genai.Client(api_key="")

In [17]:
def get_completion_from_messages(messages,system_message,
                                 model="gemini-2.5-flash-lite",
                                 temperature=0,
                                 max_tokens=500):
    # here in google gemini you have to give the input parameters as keyword arguments and not as positional arguments
    response = client.models.generate_content(model=model,
                                              contents=messages,
                                              config=types.GenerateContentConfig(system_instruction=system_message,
                                                                                temperature=temperature,
                                                                                max_output_tokens=max_tokens) 
                                             )# Pass as plain text
    return response.text  # Extract response text

In [8]:
delimiter = "####"
system_message = f"""
Follow these steps to answer the customer queries.
The customer query will be delimited with four hashtags,\
i.e. {delimiter}. 

Step 1:{delimiter} First decide whether the user is \
asking a question about a specific product or products. \
Product cateogry doesn't count. 

Step 2:{delimiter} If the user is asking about \
specific products, identify whether \
the products are in the following list.
All available products: 
1. Product: TechPro Ultrabook
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-UB100
   Warranty: 1 year
   Rating: 4.5
   Features: 13.3-inch display, 8GB RAM, 256GB SSD, Intel Core i5 processor
   Description: A sleek and lightweight ultrabook for everyday use.
   Price: $799.99

2. Product: BlueWave Gaming Laptop
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-GL200
   Warranty: 2 years
   Rating: 4.7
   Features: 15.6-inch display, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 3060
   Description: A high-performance gaming laptop for an immersive experience.
   Price: $1199.99

3. Product: PowerLite Convertible
   Category: Computers and Laptops
   Brand: PowerLite
   Model Number: PL-CV300
   Warranty: 1 year
   Rating: 4.3
   Features: 14-inch touchscreen, 8GB RAM, 256GB SSD, 360-degree hinge
   Description: A versatile convertible laptop with a responsive touchscreen.
   Price: $699.99

4. Product: TechPro Desktop
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-DT500
   Warranty: 1 year
   Rating: 4.4
   Features: Intel Core i7 processor, 16GB RAM, 1TB HDD, NVIDIA GeForce GTX 1660
   Description: A powerful desktop computer for work and play.
   Price: $999.99

5. Product: BlueWave Chromebook
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-CB100
   Warranty: 1 year
   Rating: 4.1
   Features: 11.6-inch display, 4GB RAM, 32GB eMMC, Chrome OS
   Description: A compact and affordable Chromebook for everyday tasks.
   Price: $249.99

Step 3:{delimiter} If the message contains products \
in the list above, list any assumptions that the \
user is making in their \
message e.g. that Laptop X is bigger than \
Laptop Y, or that Laptop Z has a 2 year warranty.

Step 4:{delimiter} If the user made any assumptions, \
figure out whether the assumption is true based on your \
product information. 

Step 5:{delimiter} First, politely correct the \
customer's incorrect assumptions if applicable. \
Only mention or reference products in the list of \
5 available products, as these are the only 5 \
products that the store sells. \
Answer the customer in a friendly tone.

Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 4 reasoning>
Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step.
"""

# In GEMINI you cannot pass the chatgpt's structure that uses JSON format.
# GEMINI USES ITS OWN FORMAT

In [11]:
# User query
user_message = """
by how much is the BlueWave Chromebook more expensive than the TechPro Desktop
"""

# Formatting messages
messages =  f"{delimiter}{user_message}{delimiter}"

# Get response from Gemini
response = get_completion_from_messages(messages,system_message)
print(response)

Step 1:#### The user is asking a question about specific products: the BlueWave Chromebook and the TechPro Desktop.
Step 2:#### Both the BlueWave Chromebook and the TechPro Desktop are in the provided list of products.
Step 3:#### The user is making an assumption that the BlueWave Chromebook is more expensive than the TechPro Desktop.
Step 4:#### The assumption is false. The BlueWave Chromebook costs $249.99, while the TechPro Desktop costs $999.99. Therefore, the TechPro Desktop is more expensive than the BlueWave Chromebook.
Response to user:#### Hi there! It looks like the TechPro Desktop is actually more expensive than the BlueWave Chromebook. The TechPro Desktop is priced at $999.99, while the BlueWave Chromebook is $249.99. Let me know if you have any other questions!


In [18]:
messages= {
    "role": "user", 
    "parts": [{"text": "by how much is the BlueWave Chromebook more expensive than the TechPro Desktop"}]
    # here parts is a list as the inputs from user can be multi modal and not text only
}

# Get response from Gemini
response = get_completion_from_messages(messages,system_message)
print(response)

Step 1:#### The user is asking a question about specific products: the BlueWave Chromebook and the TechPro Desktop.
Step 2:#### Both the BlueWave Chromebook and the TechPro Desktop are in the provided list of products.
Step 3:#### The user is making an assumption that the BlueWave Chromebook is more expensive than the TechPro Desktop.
Step 4:#### The assumption is false. The BlueWave Chromebook costs $249.99, while the TechPro Desktop costs $999.99. Therefore, the TechPro Desktop is more expensive than the BlueWave Chromebook.
Response to user:#### Hi there! It looks like you're asking about the pricing difference between the BlueWave Chromebook and the TechPro Desktop. Actually, the TechPro Desktop is more expensive than the BlueWave Chromebook. The BlueWave Chromebook is priced at $249.99, and the TechPro Desktop is $999.99. Let me know if you have any other questions!


4. Why is it so "complicated"?
You might wonder: "Why do I need a list called 'parts' just for one sentence?"

It’s because Gemini allows you to mix types in a single turn. In "Gemini style," a single message could look like this:

# A multimodal message (Text + Image)
messages = [{
    "role": "user",
    "parts": [
        {"text": "Look at this receipt. How much was the coffee?"},
        {"inline_data": {"mime_type": "image/jpeg", "data": image_bytes}}
    ]
}]

# Few Multi turn examples

messages = [
    {
        "role": "user",
        "parts": [{"text": "Hi, I'm looking for a new laptop."}]
    },
    {
        "role": "model",
        "parts": [{"text": "Hello! I can help with that. What is your budget?"}]
    },
    {
        "role": "user",
        "parts": [{"text": "Around $1,000. What do you recommend?"}]
    }
]

# Send the entire history to get the next response
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=messages
)

In [13]:
try:
    final_response = response.split("####")[-1].strip()
except Exception as e:
    final_response = "Sorry, I'm having trouble right now, please try asking another question."
    
print(final_response)

Hi there! It looks like the TechPro Desktop is actually more expensive than the BlueWave Chromebook. The TechPro Desktop is priced at $999.99, while the BlueWave Chromebook is $249.99. Let me know if you have any other questions!


In [14]:
def get_completion_from_messages(messages,
                                 model="gemini-2.5-flash-lite",
                                 temperature=0,
                                 max_tokens=500):
    # here in google gemini you have to give the input parameters as keyword arguments and not as positional arguments
    response = client.models.generate_content(model=model,
                                              contents=messages,
                                              config=types.GenerateContentConfig(
                                                                                temperature=temperature,
                                                                                max_output_tokens=max_tokens) 
                                             )# Pass as plain text
    return response.text  # Extract response text

In [15]:
# User query
user_message = """
by how much is the BlueWave Chromebook more expensive than the TechPro Desktop
"""

# Formatting messages
messages =  [f"{delimiter}{user_message}{delimiter}"]
messages.append(system_message)

# Get response from Gemini
response = get_completion_from_messages(messages)
print(response)

Step 1:#### The user is asking a question about two specific products: the BlueWave Chromebook and the TechPro Desktop.
Step 2:#### Both the BlueWave Chromebook and the TechPro Desktop are in the provided list of available products.
Step 3:#### The user is assuming that the BlueWave Chromebook is more expensive than the TechPro Desktop.
Step 4:#### The assumption is false. The BlueWave Chromebook costs $249.99, and the TechPro Desktop costs $999.99. Therefore, the TechPro Desktop is more expensive than the BlueWave Chromebook.
Response to user:#### Hello there! To answer your question, the BlueWave Chromebook is actually less expensive than the TechPro Desktop. The BlueWave Chromebook is priced at $249.99, while the TechPro Desktop is $999.99. Let me know if you have any other questions!


In [16]:
messages

['####\nby how much is the BlueWave Chromebook more expensive than the TechPro Desktop\n####',
 "\nFollow these steps to answer the customer queries.\nThe customer query will be delimited with four hashtags,i.e. ####. \n\nStep 1:#### First decide whether the user is asking a question about a specific product or products. Product cateogry doesn't count. \n\nStep 2:#### If the user is asking about specific products, identify whether the products are in the following list.\nAll available products: \n1. Product: TechPro Ultrabook\n   Category: Computers and Laptops\n   Brand: TechPro\n   Model Number: TP-UB100\n   Warranty: 1 year\n   Rating: 4.5\n   Features: 13.3-inch display, 8GB RAM, 256GB SSD, Intel Core i5 processor\n   Description: A sleek and lightweight ultrabook for everyday use.\n   Price: $799.99\n\n2. Product: BlueWave Gaming Laptop\n   Category: Computers and Laptops\n   Brand: BlueWave\n   Model Number: BW-GL200\n   Warranty: 2 years\n   Rating: 4.7\n   Features: 15.6-inch d

In [ ]:
# In GEMINI you cannot pass the chatgpt's structure that uses JSON format.
# GEMINI USES ITS OWN FORMAT